In [18]:
from datasets import load_titanic, balance_classes
from tools import model_performance,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_roc_curve
from pygam import LogisticGAM, s, f, l
import numpy as np
np.set_printoptions(suppress=True)

In [19]:
df, df_train, df_test = load_titanic()
df_train.columns

Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'target'], dtype='object')

In [20]:
original_X = df.drop(['target'], axis=1)
original_y = df['target']
train_X = df_train.drop(['target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['target'], axis=1)
test_y = df_test['target']

In [21]:
original_X = pd.get_dummies(original_X)
train_X = pd.get_dummies(train_X)
test_X = pd.get_dummies(test_X)

features = train_X.columns

In [22]:
scaler = StandardScaler().fit(original_X)
original_X = scaler.transform(original_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

original_X = pd.DataFrame(original_X, columns=features)
train_X = pd.DataFrame(train_X, columns=features)
test_X = pd.DataFrame(test_X, columns=features)

## Pocet komponent

In [23]:
pca = PCA(n_components=len(features)-1)
pca = pca.fit(original_X)

In [24]:
pca.explained_variance_ratio_

array([0.2479778 , 0.19668335, 0.16836786, 0.13343106, 0.09408205,
       0.06749907, 0.05611914, 0.03583969, 0.        ])

## Vizualizace

In [25]:
pca = PCA(n_components=3)
pca = pca.fit(original_X)

In [26]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [27]:
transformed_df = pd.DataFrame(transformed_original_X, columns=['x','y','z'])
transformed_df['target'] = original_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target')

## Classification

In [28]:
pca = PCA(n_components=4)
pca = pca.fit(original_X)

In [29]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [30]:
model = LogisticRegression(penalty='none')

In [31]:
# stratified (balanced), 5-fold - zadne zlepseni oproti ciste LogisticRegression
cross_val_score(model, transformed_original_X, original_y).mean()

0.7811562362689097

In [32]:
model = model.fit(transformed_train_X, train_y)

In [33]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.7865168539325843
Test score
0.7947761194029851


In [34]:
confusion_matrix(model.predict(transformed_train_X), train_y, model.predict(transformed_test_X), test_y)

Train
TN: 327, TP: 163, FN: 76, FP: 57
Test
TN: 143, TP: 70, FN: 33, FP: 22
